In [2]:
from pathlib import Path
import json

# load COCO metadata
filename = "data/captions_train2014.json"
with Path(filename).open() as f:
    coco_data = json.load(f)

In [39]:
class coco_values:
    def __init__(self, coco_data):
        self.coco_annotations = coco_data['annotations']
        self.coco_images = coco_data['images']
        self.coco_img_ids = [i['id'] for i in self.coco_images]
        self.coco_cap_ids = [i['id'] for i in self.coco_annotations]
    def get_captions(self, image_id):
        return [i['id'] for i in self.coco_annotations if i['image_id'] == image_id]
    def get_image_from_cap(self, caption_id):
        return [i['image_id'] for i in self.coco_annotations if i['id'] == caption_id][0]
    def get_caption_text(self, caption_id):
        return [i['caption'] for i in self.coco_annotations if i['id'] == caption_id][0]

In [40]:
x = coco_values(coco_data)

In [41]:
x.get_captions(318556)

[48, 126, 219, 255, 3555]

In [42]:
x.get_image_from_cap(48)

318556

In [43]:
x.get_caption_text(48)

'A very clean and well decorated empty bathroom'